In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Install keras tuner everytime you restart colab

! pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.8 MB/s eta 0:00:00


In [ ]:
import keras_tuner

from kerastuner.tuners import RandomSearch

In [ ]:
data =pd.read_csv('/content/drive/MyDrive/AQI (1).csv')
data

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [ ]:
data.head(3)

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500


In [ ]:
# Independed feature and depended feature

x= data.iloc[:, :-1]
y= data.iloc[:, -1]

In [ ]:
# train test split

x_train, x_test, y_train, y_test= train_test_split( x,y, test_size=0.2, random_state=42)

In [ ]:
# Standerrized data

scale=StandardScaler()

x_train_scaled= scale.fit_transform(x_train)
x_test_scaled = scale.transform(x_test)

In [ ]:
def build_model(hp):  # class

  # initalize model
  model = Sequential()

  # 1) input layer
  model.add(Dense(units=8, kernel_initializer='he_uniform', input_dim=8))

  # 2) hidden layer
  for i in range(hp.Int('num_layers', 2,5)):
      model.add(Dense( units= hp.Int('units_'+str(i), min_value=2, max_value=6, step=2), kernel_initializer='he_uniform',
                      activation=hp.Choice('activation', ['relu','linear', 'tanh']) ))

  # 3) output layer

  model.add(Dense(units=1, activation='relu'))


  # Backword Propogation

  model.compile( optimizer= keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                loss='mean_squared_error', metrics=['accuracy'] )

  return model



In [ ]:
# Spin tuner

tuner= RandomSearch(hypermodel=build_model,
                    objective='val_accuracy',
                    max_trials=3,
                    executions_per_trial=2,
                    directory='my_dir',
                    project_name='AQI')

Reloading Tuner from my_dir/AQI/tuner0.json


In [ ]:
# tuner summary

tuner.search_space_summary()

Search space summary
Default search space size: 7
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'linear', 'tanh'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}


In [ ]:
# fit data into tuner

tuner.search(x_train_scaled, y_train, epochs=5, validation_data=(x_test_scaled, y_test))

Trial 3 Complete [00h 00m 09s]
val_accuracy: 0.006849315017461777

Best val_accuracy So Far: 0.006849315017461777
Total elapsed time: 00h 05m 00s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/AQI
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 5
units_0: 4
activation: tanh
units_1: 6
learning_rate: 0.0001
units_2: 6
units_3: 2
units_4: 2
Score: 0.006849315017461777

Trial 1 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: relu
units_1: 4
learning_rate: 0.0001
units_2: 4
units_3: 6
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = s


Trial 2 summary
Hyperparameters:
num_layers: 5
units_0: 4
activation: tanh
units_1: 6
learning_rate: 0.0001
units_2: 6
units_3: 2
units_4: 2
Score: 0.006849315017461777
****
Trial 1 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: relu
units_1: 4
learning_rate: 0.0001
units_2: 4
units_3: 6
***
Trial 0 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: tanh
units_1: 2
learning_rate: 0.0001
units_2: 2
units_3: 2

In [ ]:

Trial 2 summary
Hyperparameters:
num_layers: 5
units_0: 4
activation: tanh
units_1: 6
learning_rate: 0.0001
units_2: 6
units_3: 2
units_4: 2
Score: 0.006849315017461777
****
Trial 1 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: relu
units_1: 4
learning_rate: 0.0001
units_2: 4
units_3: 6
***
Trial 0 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: tanh
units_1: 2
learning_rate: 0.0001
units_2: 2
units_3: 2